In [3]:
# Assumes that the server is running
from openai import OpenAI

arbor_port = 7453

client = OpenAI(
    base_url=f"http://127.0.0.1:{arbor_port}/v1",  # Using Arbor server
    api_key="not-needed",  # If you're using a local server, you dont need an API key
)


response = client.chat.completions.create(
    model="Qwen/Qwen2-0.5B-Instruct",
    messages=[
        {"role": "user", "content": "Hello, how are you?"},
    ],
    temperature=0.7,
)
print(response)




ChatCompletion(id='chatcmpl-71084696cb9b49ccafc2020e584ef5db', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1751050857, model='Qwen/Qwen2-0.5B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None)


In [48]:
import requests
import json

arbor_port = 7453

def upload_file_oai(file_path):
    return client.files.create(
        file=open(file_path, 'rb'),
        purpose='fine-tune'
    )

def upload_file(file_path, url=f'http://127.0.0.1:{arbor_port}/v1/files'):
    """
    Upload a file to the specified URL endpoint.

    Args:
        file_path (str): Path to the file to be uploaded
        url (str): URL endpoint for file upload (default: http://127.0.0.1:{arbor_port}/api/files)

    Returns:
        requests.Response: Response from the server
    """
    with open(file_path, 'rb') as file:
        files = {'file': file}
        response = requests.post(url, files=files)
    return response

# Example usage:
response = upload_file('../tests/data/training_data_sft.jsonl')
# response = upload_file_oai('../tests/data/training_data_sft.jsonl')
print(response.status_code)  # Print the HTTP status code
response_body = json.loads(response.text)
uploaded_file = response_body["id"]
print(uploaded_file)

200
file-fbe13b80-66d5-4286-98c0-de3d901bf7e8


In [52]:
#"HuggingFaceTB/SmolLM2-135M-Instruct"
#"Qwen/Qwen2-0.5B-Instruct"
def start_fine_tune(training_file, model_name="Qwen/Qwen2-0.5B-Instruct"):
    return client.fine_tuning.jobs.create(
        training_file=training_file,
        model=model_name,
    )

# Example usage (continuing from previous upload):
fine_tune_response = start_fine_tune(uploaded_file)
print(fine_tune_response)

FineTuningJob(id='ftjob-29e19b7a-cf8d-4532-86ce-9e87edf3434e', created_at=None, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=None, model=None, object='fine_tuning.job', organization_id=None, result_files=None, seed=None, status='queued', trained_tokens=None, training_file=None, validation_file=None, estimated_finish=None, integrations=None, metadata=None, method=None)


In [56]:
def check_job_status_oai(job_id):
    return client.fine_tuning.jobs.retrieve(job_id)


def check_job_status(job_id, url_base=f'http://127.0.0.1:7453/v1/fine_tuning/jobs'):
    """
    Check the status of a fine-tuning job.

    Args:
        job_id (str): ID of the fine-tuning job
        url_base (str): Base URL for job status endpoint (default: http://127.0.0.1:8000/api/job)

    Returns:
        requests.Response: Response from the server
    """
    url = f"{url_base}/{job_id}"
    response = requests.get(url)
    return response

# Example usage (continuing from previous fine-tune):
job_response = check_job_status(fine_tune_response.id)
print(job_response.status_code)
print(job_response.text)
job_response

200
{"object":"fine_tuning.job","id":"ftjob-29e19b7a-cf8d-4532-86ce-9e87edf3434e","fine_tuned_model":null,"status":"queued"}


<Response [200]>